# 1. importar librerías y settings

In [ ]:
# importar librerías
import os, pandas as pd
import matplotlib.pyplot as plt

# Importar UDFs
import plotting_udfs as pf

# comprobar WD
print(os.getcwd())

# ampliar visualizaciones de pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

***
# 2.flowchart

In [ ]:
# exportar stats
stats_path="./Anexo_5_carpeta_1_resultados/tabla_5_stats.csv"
df=pd.read_csv(stats_path, sep=";")

# castear columnas a tipo int
cols = list(df.columns[1:])
for col in cols:
    df[col] = df[col].astype('Int64')

###############
# INIT.SETTINGS
###############
# Crear figura y ejes
fig, ax = plt.subplots(figsize=(10, 12))

# Crear cajas para cada etapa con tamaños ajustados
box_width = 4


############
# TEXT BOXES
############
# Identificación
id_text = f"{df.iloc[5,0]}\n\n(n={df.iloc[5,4]})"
pf.create_box(ax, 5, 9.5, box_width, 1.2, id_text, max_chars=40)
id_detail = f"PubMed (n={df.iloc[5,1]})\n\nWeb of Science (n={df.iloc[5,2]})"
pf.create_box(ax, 9.5, 9.5, 3.8, 1.5, id_detail, max_chars=40)

# incluidos: pre-Screening
ps_text = f"Registros incluidos tras\nfiltrado sistemático pre-screening\n\n(n={df.iloc[10,4]})"
pf.create_box(ax, 5, 7.5, box_width, 1.2, ps_text, max_chars=45)

# excluidos: pre-screening
ex_psc = df.iloc[6,3] + df.iloc[7,3] + df.iloc[8,3] + df.iloc[9,3] + df.iloc[10,3]
eb1_text = f"Excluidos pre-screening: {ex_psc}\n\n{df.iloc[6,0]} (n={df.iloc[6,3]})\n{df.iloc[7,0]} (n={df.iloc[7,3]})\n{df.iloc[8,0]} (n={df.iloc[8,3]})\n{df.iloc[9,0]} (n={df.iloc[9,3]})\n{df.iloc[10,0]} (n={df.iloc[10,3]})"
pf.create_box(ax, 9.5, 7.5, 3.8, 1.5, eb1_text, max_chars=40, align_bullets=True)

# incluidos: tiab screening
sc_text = f"Artículos incluidos tras\nscreening de títulos y resúmenes\n\n(n={df.iloc[11,4]})"
pf.create_box(ax, 5, 5.5, box_width, 1.2, sc_text, max_chars=40)

# excluidos: tiab screening
excluded_tiab_screening = f"{df.iloc[12,0]} (n={df.iloc[12,3]})\n{df.iloc[13,0]} (n={df.iloc[13,3]})\n{df.iloc[14,0]} (n={df.iloc[14,3]})\n{df.iloc[15,0]} (n={df.iloc[15,3]})"
eb2_text = f"Excluidos screening: {df.iloc[11,3]}\n\n{excluded_tiab_screening}"
pf.create_box(ax, 9.5, 5.5, 3.8, 1.2, eb2_text, max_chars=40, align_bullets=True)

# incluidos: ft review
ft_text = f"Estudios incluidos\nen síntesis\n\n(n={df.iloc[16,4]})"
pf.create_box(ax, 5, 3.5, box_width, 1.2, ft_text, max_chars=40)

# excluidos: ft review
excluded_full_text = f"{df.iloc[17,0]} (n={df.iloc[17,3]})\n{df.iloc[18,0]} (n={df.iloc[18,3]})\n{df.iloc[19,0]} (n={df.iloc[19,3]})\n{df.iloc[20,0]} (n={df.iloc[20,3]})"
eb3_text = f"Excluidos full-text screening: {df.iloc[16,3]}\n\n{excluded_full_text}"
exclusion_box3 = pf.create_box(ax, 9.5, 3.5, 3.8, 1.2, eb3_text, max_chars=40, align_bullets=True)

# incluidos detalle
in_detail = f"Tipos de estudios incluidos en la síntesis cualitativa:\n\n• {df.iloc[22,0]} (n={df.iloc[22,4]})\n• {df.iloc[23,0]} (n={df.iloc[23,4]})\n• {df.iloc[24,0]} (n={df.iloc[24,4]})\n• {df.iloc[25,0]} (n={df.iloc[26,4]})\n• {df.iloc[26,0]} (n={df.iloc[26,4]})"
pf.create_box(ax, 5, 1.5, box_width, 1.2, in_detail, max_chars=40, align_bullets=True)


################
# FLECHAS Y PLOT
################
# Crear flechas verticales conectoras
for y_start, y_end in [(9.0, 8.05), (7.0, 6.05), (5.0, 4.05), (3.0, 2.05)]:
    pf.create_vertical_arrow(ax, 5, y_start, y_end)

# Crear flechas horizontales para conexiones laterales
for y_pos in [9.5, 7.5, 5.5, 3.5]:
    pf.create_horizontal_arrow(ax, 7.0, 7.65, y_pos)

# Configuración general
plt.axis('off')
plt.xlim(1, 13)
plt.ylim(0.5, 10.5)
#plt.title('Diagrama de Flujo PRISMA', fontsize=18, fontweight='bold')
plt.tight_layout()

# Guardar
plt.savefig('./Anexo_5_carpeta_1_resultados/fig1_flowchart.png', dpi=300, bbox_inches='tight')
#plt.savefig('./fig3.1_prisma_diagram.pdf', bbox_inches='tight')  # También en PDF para mejor calidad
plt.show()

***
# 3. barplots
### 3.1.omics_techniques

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from collections import Counter

# Función simplificada para normalizar texto (respetando acentos)
def normalize_text(text):
    if not isinstance(text, str):
        return text
    
    # Convertir a minúsculas y eliminar espacios extras
    text = text.lower().strip()
    text = ' '.join(text.split())
    
    return text

# Cargar datos
df = pd.read_csv("./Anexo_5_carpeta_1_resultados/tabla_3_articles.csv", sep=";")
data = df["bullets_omicas"].tolist()

# Extraer ómicas y técnicas
omics, techniques = pf.extract_omics_and_techniques(data)

# Crear diccionarios case-insensitive para contar frecuencias
omics_counter = {}
for omic in omics:
    key = normalize_text(omic)
    if key not in omics_counter:
        omics_counter[key] = {'original': omic, 'count': 0}
    omics_counter[key]['count'] += 1

techniques_counter = {}
for technique in techniques:
    key = normalize_text(technique)
    if key not in techniques_counter:
        techniques_counter[key] = {'original': technique, 'count': 0}
    techniques_counter[key]['count'] += 1

# Crear DataFrames para visualización usando los valores originales (pero contados correctamente)
omics_df = pd.DataFrame({
    'Ómica': [info['original'] for info in omics_counter.values()],
    'Frecuencia': [info['count'] for info in omics_counter.values()]
})

techniques_df = pd.DataFrame({
    'Técnica': [info['original'] for info in techniques_counter.values()],
    'Frecuencia': [info['count'] for info in techniques_counter.values()]
})

# Palabras que deben permanecer en minúscula (excepto al inicio de frase)
lowercase_words = ["de", "y", "con", "a", "en", "para", "por", "del"]

# Palabras que tienen un formato específico
special_words = ["RMN", "RNA-seq", "RT-PCR", "qRT-PCR", "GWAS", "snRNA-seq", "NGS", "MS"]

# Aplicar el formato a la columna "Ómica"
omics_df['Ómica'] = omics_df['Ómica'].apply(
    lambda x: pf.format_string(x, lowercase_words, special_words)
)

## cambiar valor de "redes de interacción" para mejor representación
c = omics_df["Ómica"] == "Análisis de Predicción y Construcción de Redes de Interacción"
omics_df.loc[c, "Ómica"] = "Análisis de Predicción y Construcción\nde Redes de Interacción"

# Aplicar el formato a la columna "Técnica"
techniques_df['Técnica'] = techniques_df['Técnica'].apply(
    lambda x: pf.format_string(x, lowercase_words, special_words)
)

# Ordenar para la visualización (de mayor a menor, pero invertido para el gráfico)
omics_df = omics_df.sort_values('Frecuencia', ascending=True)
techniques_df = techniques_df.sort_values('Frecuencia', ascending=True)

# Configurar estilo
sns.set(style="whitegrid")
plt.figure(figsize=(15, 20))

# Variables de configuración
otitle = "Número de estudios según campos ómicos y técnicas bioinformáticas"
ttitle = "Número de estudios según técnicas concretas"
xfs = 18
bfs = xfs

# Generar paletas de colores
colors_omics = sns.color_palette("viridis", len(omics_df))
colors_techniques = sns.color_palette("magma", len(techniques_df))

# Gráfico para Ómicas (arriba)
plt.subplot(2, 1, 1)
ax1 = plt.gca()
y_pos = range(len(omics_df))
plt.barh(y=y_pos, width=omics_df['Frecuencia'], color=colors_omics)
plt.yticks(y_pos, omics_df['Ómica'])
plt.title('')
plt.xlabel(otitle, fontsize=xfs)
plt.ylabel('')
plt.tick_params(axis='both', which='major', labelsize=bfs)
plt.grid(axis='x', linestyle='--', alpha=0.7)
ax1.xaxis.set_major_locator(ticker.MaxNLocator(integer=True)) # Configurar el eje x para mostrar solo enteros

# Gráfico para Técnicas (abajo)
plt.subplot(2, 1, 2)
ax2 = plt.gca()
y_pos = range(len(techniques_df))
plt.barh(y=y_pos, width=techniques_df['Frecuencia'], color=colors_techniques)
plt.yticks(y_pos, techniques_df['Técnica'])
plt.title('')
plt.xlabel(ttitle, fontsize=xfs)
plt.ylabel('')
plt.tick_params(axis='both', which='major', labelsize=bfs)
plt.grid(axis='x', linestyle='--', alpha=0.7)
ax2.xaxis.set_major_locator(ticker.MaxNLocator(integer=True)) # Configurar el eje x para mostrar solo enteros

# Ajustar el espacio entre los gráficos
igs = 0.3  # Reducido de 6 para un espaciado más razonable
plt.subplots_adjust(hspace=igs)
plt.tight_layout()

# Guardar gráficos
plt.savefig('./Anexo_5_carpeta_1_resultados/fig2_techniques_barplot.png', dpi=300, bbox_inches='tight')
plt.show()

# Mostrar también las tablas de frecuencia
print("Tabla: número de estudios según campos ómicos y técnicas bioinformáticas")
print(omics_df.sort_values('Frecuencia', ascending=False))  # Invertir para mostrar de mayor a menor
print("\nTabla: número de estudios según técnicas concretas")
print(techniques_df.sort_values('Frecuencia', ascending=False))  # Invertir para mostrar de mayor a menor

***
### 3.2.biomarkers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

# Función simplificada para normalizar texto (respetando acentos)
def normalize_text(text):
    if not isinstance(text, str):
        return text
    
    # Convertir a minúsculas y eliminar espacios extras
    text = text.lower().strip()
    text = ' '.join(text.split())
    
    return text


# Función para procesar la columna type_sample_tech
def process_type_sample(text):
    if not isinstance(text, str):
        return [], []
    
    # Dividir por el primer slash y tomar solo la primera parte
    before_slash = text.split('/', 1)[0].strip()
    
    # Extraer el texto dentro de los paréntesis
    sample_match = re.search(r'\((.*?)\)', before_slash)
    sample = sample_match.group(1) if sample_match else ""
    
    # Extraer el texto fuera de los paréntesis
    type_text = re.sub(r'\(.*?\)', '', before_slash).strip()
    
    # Manejar múltiples tipos (separados por coma)
    types = [t.strip() for t in type_text.split(',')]
    
    # Manejar múltiples muestras (separadas por coma)
    samples = [s.strip() for s in sample.split(',')]
    
    return types, samples


# Función para formatear strings
def format_string(text, lowercase_words, special_words):
    
    # Primero aplicamos title() a todo el texto
    text_titled = text.title()
    
    # Dividimos en palabras
    words = text_titled.split()
    
    # Procesamos palabra por palabra
    for i, word in enumerate(words):
        
        # Si la palabra está en lowercase_words y no es la primera palabra, la ponemos en minúsculas
        if word.lower() in lowercase_words and i > 0:
            words[i] = word.lower()
        
        # Si la palabra está en special_words, usamos esa versión exacta
        elif word.upper() in [w.upper() for w in special_words]:
            for special in special_words:
                if word.upper() == special.upper():
                    words[i] = special
                    break
    
    # Unimos nuevamente las palabras
    return ' '.join(words)

# Cargar y checkear los datos
df = pd.read_csv("./Anexo_5_carpeta_1_resultados/tabla_4_biomarkers.csv", sep=";")  # Ajusta el nombre del archivo
df["type_sample_tech"].head()

In [ ]:
# Extraer tipos y muestras
types_samples = df['type_sample_tech'].apply(process_type_sample)
types_samples.head()

In [ ]:
# asignar elementos de type_samples a distintas columnas
df['tipos'] = types_samples.apply(lambda x: x[0])
df['muestras'] = types_samples.apply(lambda x: x[1])

# Expandir las listas
all_types = []
for types_list in df['tipos']:
    all_types.extend(types_list)

all_samples = []
for samples_list in df['muestras']:
    all_samples.extend(samples_list)

# Crear un diccionario case-insensitive para contar frecuencias
biomarcador_counter = {}
for tipo in all_types:
    key = normalize_text(tipo)  # Clave en minúsculas para comparación
    if key not in biomarcador_counter:
        biomarcador_counter[key] = {'original': tipo, 'count': 0}
    biomarcador_counter[key]['count'] += 1

muestra_counter = {}
for muestra in all_samples:
    key = normalize_text(muestra)  # Clave en minúsculas para comparación
    if key not in muestra_counter:
        muestra_counter[key] = {'original': muestra, 'count': 0}
    muestra_counter[key]['count'] += 1

# Crear DataFrames usando los valores originales (pero contados correctamente)
biomarcador_df = pd.DataFrame({
    'Tipo de biomarcador': [info['original'] for info in biomarcador_counter.values()],
    'Frecuencia': [info['count'] for info in biomarcador_counter.values()]
})

muestra_df = pd.DataFrame({
    'Muestra': [info['original'] for info in muestra_counter.values()],
    'Frecuencia': [info['count'] for info in muestra_counter.values()]
})

# check
muestra_df

In [ ]:
import matplotlib.ticker as ticker

# Palabras que deben permanecer en minúscula (excepto al inicio de frase)
lowercase_words = ["de", "y", "con", "a", "en", "para", "por", "del", "la"]

# Palabras que tienen un formato específico
special_words = ["RMN", "RNA-seq", "RT-PCR", "qRT-PCR", "GWAS", "snRNA-seq", "NGS", "MS", "VE", "NMR", "DXA", "ELISA", "PCR"]

# Aplicar formato a las columnas
biomarcador_df['Tipo de biomarcador'] = biomarcador_df['Tipo de biomarcador'].apply(
    lambda x: format_string(x, lowercase_words, special_words)
)

muestra_df['Muestra'] = muestra_df['Muestra'].apply(
    lambda x: format_string(x, lowercase_words, special_words)
)

# Ordenar para la visualización
biomarcador_df = biomarcador_df.sort_values('Frecuencia', ascending=True)
muestra_df = muestra_df.sort_values('Frecuencia', ascending=True)

In [ ]:
# Configurar estilo
sns.set(style="whitegrid")
plt.figure(figsize=(15, 20))

# Títulos para los ejes x
btitle = "Número de biomarcadores según tipo"
mtitle = "Número de biomarcadores según muestra"
xfs = 18
bfs = xfs

# Generar paletas de colores
colors_biomarker = sns.color_palette("viridis", len(biomarcador_df))
colors_sample = sns.color_palette("magma", len(muestra_df))

# Gráfico para Tipos de biomarcador (arriba)
plt.subplot(2, 1, 1)
ax1 = plt.gca()  # Obtener el eje actual
y_pos = range(len(biomarcador_df))
plt.barh(y=y_pos, width=biomarcador_df['Frecuencia'], color=colors_biomarker)
plt.yticks(y_pos, biomarcador_df['Tipo de biomarcador'])
plt.title('')
plt.xlabel(btitle, fontsize=xfs)
plt.ylabel('')
plt.tick_params(axis='both', which='major', labelsize=bfs)
plt.grid(axis='x', linestyle='--', alpha=0.7)
ax1.xaxis.set_major_locator(ticker.MaxNLocator(integer=True)) # Configurar el eje x para mostrar solo enteros

# Gráfico para Muestras (abajo)
plt.subplot(2, 1, 2)
ax2 = plt.gca()  # Obtener el eje actual
y_pos = range(len(muestra_df))
plt.barh(y=y_pos, width=muestra_df['Frecuencia'], color=colors_sample)
plt.yticks(y_pos, muestra_df['Muestra'])
plt.title('')
plt.xlabel(mtitle, fontsize=xfs)
plt.ylabel('')
plt.tick_params(axis='both', which='major', labelsize=bfs)
plt.grid(axis='x', linestyle='--', alpha=0.7)
ax1.xaxis.set_major_locator(ticker.MaxNLocator(integer=True)) # Configurar el eje x para mostrar solo enteros
plt.subplots_adjust(hspace=0.3)
plt.tight_layout()

# Guardar gráficos
plt.savefig('./Anexo_5_carpeta_1_resultados/fig3_biomarkers_barplot.png', dpi=300, bbox_inches='tight')
plt.show()

# Mostrar las tablas de frecuencia finales
print("\nTabla: Número de biomarcadores según tipo")
print(biomarcador_df.sort_values('Frecuencia', ascending=False))
print("\nTabla: Número de biomarcadores según muestra")
print(muestra_df.sort_values('Frecuencia', ascending=False))